In [135]:
import pandas as pd

## Loading Datasets


In [136]:
jobs = pd.read_csv("../data/job_postings.csv", engine='python', on_bad_lines='skip', dtype=str)
skills=pd.read_excel("../data/skills.xlsx")

In [137]:
jobs.drop("Unnamed: 0",axis=1,inplace=True)


In [138]:
# Making Column in Proper Manner
jobs.columns = jobs.columns.str.replace(" ", "_").str.lower()


## Checking DataTypes

In [139]:
jobs.dtypes

job_title            object
salary_estimate      object
job_description      object
rating               object
company_name         object
location             object
headquarters         object
size                 object
founded              object
type_of_ownership    object
industry             object
sector               object
revenue              object
competitors          object
easy_apply           object
dtype: object

##  Checking Null Values


In [140]:
jobs.isnull().sum()

job_title            0
salary_estimate      0
job_description      0
rating               0
company_name         1
location             0
headquarters         0
size                 0
founded              0
type_of_ownership    0
industry             0
sector               0
revenue              0
competitors          0
easy_apply           0
dtype: int64

## Checking Duplicacy

In [141]:
jobs.duplicated().sum()

np.int64(0)

## Data Cleaning

In [142]:
jobs['job_description'] = (
    jobs['job_description']
        .astype(str)
        .str.lower()
        .str.replace(r'[^a-zA-Z ]', ' ', regex=True)
)


In [143]:
jobs['easy_apply'] = (
    jobs['easy_apply']
        .astype(str)
        .str.strip()
        .str.upper()
        .map({
            'TRUE': 1,
            '1': 1,
            '-1': 0,
            'FALSE': 0,
            '0': 0
        })
        .astype('Int8')   # supports NA
)



In [144]:
jobs["founded"]=jobs["founded"].replace(-1,pd.NA)
jobs['founded'] = pd.to_numeric(jobs['founded'], errors='coerce')


In [145]:
jobs = jobs[jobs['salary_estimate'] != '-1'].copy()

jobs['salary_clean'] = jobs['salary_estimate'].str.replace(r'\(.*?\)', '', regex=True)
jobs['salary_clean'] = jobs['salary_clean'].str.replace('$', '', regex=False)
jobs['salary_clean'] = jobs['salary_clean'].str.replace('K', '', regex=False)
jobs['salary_clean'] = jobs['salary_clean'].str.replace(',', '', regex=False)  # remove commas

salary_split = jobs['salary_clean'].str.split('-', expand=True)
jobs['min_salary'] = salary_split[0].astype(int)
jobs['max_salary'] = salary_split[1].fillna(salary_split[0]).astype(int)
jobs['avg_salary'] = (jobs['min_salary'] + jobs['max_salary']) / 2

jobs['salary_level'] = pd.cut(
    jobs['avg_salary'],
    bins=[0,50,80,120,200],
    labels=['Low','Medium','High','Very High']
)
jobs

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply,salary_clean,min_salary,max_salary,avg_salary,salary_level
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,1,37-66,37,66,51.5,Medium
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),overview provides analytical and technical su...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,37-66,37,66,51.5,Medium
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),we re looking for a senior data analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,0,37-66,37,66,51.5,Medium
3,Data Analyst,$37K-$66K (Glassdoor est.),requisition numberrr remote yes we col...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,0,37-66,37,66,51.5,Medium
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),about fanduel group fanduel group is a world ...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,1,37-66,37,66,51.5,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1,0,78-104,78,104,91.0,High
2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),position senior data analyst corporate audit...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,Wholesale,Business Services,$10+ billion (USD),"Avnet, Ingram Micro, Tech Data",0,78-104,78,104,91.0,High
2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),title technical business analyst sql data a...,-1,Spiceorb,"Denver, CO",-1,-1,-1,-1,-1,-1,-1,-1,0,78-104,78,104,91.0,High
2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),summary responsible for working cross functio...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD),-1,0,78-104,78,104,91.0,High


## Descriptive Statistics


In [146]:
jobs.describe()

,founded,easy_apply,min_salary,max_salary,avg_salary
count,2252.000000,2252.0,2252.000000,2252.000000,2252.000000
mean,1398.255329,0.035524,54.266874,89.979130,72.123002
std,902.040151,0.185141,19.575375,29.315605,23.600734
min,-1.000000,0.0,24.000000,38.000000,33.500000
25%,-1.000000,0.0,41.000000,70.000000,58.000000
50%,1979.000000,0.0,50.000000,87.000000,69.000000
75%,2002.000000,0.0,64.000000,104.000000,80.500000
max,2019.000000,1.0,113.000000,190.000000,150.000000


## Detect Skills in Job Descriptions

In [147]:
skill_list=skills["skill_name"].tolist()

for skill in skill_list:
   jobs[skill]=jobs["job_description"].str.contains(skill)
jobs

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,...,avg_salary,salary_level,python,sql,excel,power bi,tableau,statistics,machine learning,communication
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,...,51.5,Medium,True,True,False,False,False,True,False,False
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),overview provides analytical and technical su...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,...,51.5,Medium,False,True,True,False,False,True,False,True
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),we re looking for a senior data analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,...,51.5,Medium,True,True,True,False,True,False,False,False
3,Data Analyst,$37K-$66K (Glassdoor est.),requisition numberrr remote yes we col...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,...,51.5,Medium,False,True,False,False,True,False,False,True
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),about fanduel group fanduel group is a world ...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,...,51.5,Medium,True,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,...,91.0,High,False,False,False,False,False,False,False,False
2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),position senior data analyst corporate audit...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,...,91.0,High,False,True,False,False,True,True,False,True
2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),title technical business analyst sql data a...,-1,Spiceorb,"Denver, CO",-1,-1,-1,-1,...,91.0,High,False,True,True,False,False,False,False,True
2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),summary responsible for working cross functio...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,...,91.0,High,True,True,False,False,False,False,True,False


## Convert True/False to 1/0

In [148]:
skill_cols = [
    'python','sql','excel','power bi',
    'tableau','statistics','machine learning','communication'
]

jobs[skill_cols] = jobs[skill_cols].astype(int)
jobs

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,...,avg_salary,salary_level,python,sql,excel,power bi,tableau,statistics,machine learning,communication
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,...,51.5,Medium,1,1,0,0,0,1,0,0
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),overview provides analytical and technical su...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,...,51.5,Medium,0,1,1,0,0,1,0,1
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),we re looking for a senior data analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,...,51.5,Medium,1,1,1,0,1,0,0,0
3,Data Analyst,$37K-$66K (Glassdoor est.),requisition numberrr remote yes we col...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,...,51.5,Medium,0,1,0,0,1,0,0,1
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),about fanduel group fanduel group is a world ...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,...,51.5,Medium,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,...,91.0,High,0,0,0,0,0,0,0,0
2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),position senior data analyst corporate audit...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,...,91.0,High,0,1,0,0,1,1,0,1
2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),title technical business analyst sql data a...,-1,Spiceorb,"Denver, CO",-1,-1,-1,-1,...,91.0,High,0,1,1,0,0,0,0,1
2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),summary responsible for working cross functio...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,...,91.0,High,1,1,0,0,0,0,1,0


In [149]:
jobs.rename(columns={
    "power bi": "power_bi",
    "machine learning": "machine_learning",
    "sql": "sql_"
}, inplace=True)
jobs

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,...,avg_salary,salary_level,python,sql_,excel,power_bi,tableau,statistics,machine_learning,communication
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,...,51.5,Medium,1,1,0,0,0,1,0,0
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),overview provides analytical and technical su...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,...,51.5,Medium,0,1,1,0,0,1,0,1
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),we re looking for a senior data analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,...,51.5,Medium,1,1,1,0,1,0,0,0
3,Data Analyst,$37K-$66K (Glassdoor est.),requisition numberrr remote yes we col...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,...,51.5,Medium,0,1,0,0,1,0,0,1
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),about fanduel group fanduel group is a world ...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,...,51.5,Medium,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,...,91.0,High,0,0,0,0,0,0,0,0
2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),position senior data analyst corporate audit...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,...,91.0,High,0,1,0,0,1,1,0,1
2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),title technical business analyst sql data a...,-1,Spiceorb,"Denver, CO",-1,-1,-1,-1,...,91.0,High,0,1,1,0,0,0,0,1
2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),summary responsible for working cross functio...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,...,91.0,High,1,1,0,0,0,0,1,0


## Calculate Skill Demand Count

In [150]:
skill_list = ['python', 'sql_', 'excel', 'power_bi', 'tableau', 'statistics', 'machine_learning', 'communication']
skill_demand = jobs[skill_list].sum().reset_index()
skill_demand.columns = ['skill', 'demand_count']


In [151]:
skill_demand

,skill,demand_count
0,python,637
1,sql_,1388
2,excel,1353
3,power_bi,181
4,tableau,620
5,statistics,563
6,machine_learning,183
7,communication,1154


## Saving

In [152]:
jobs.to_csv("jobs_with_skills.csv", index=False)
skill_demand.to_csv("skill_demand_summary.csv", index=False)